In [ ]:
!pip install prophet==1.0.1

In [ ]:
# Standard imports
import pandas as pd
import math
from datetime import datetime, timedelta
import numpy as np
import sys

# Prophet imports
from prophet import Prophet
from prophet.plot import (plot_plotly, plot_components_plotly, 
                           plot_forecast_component)
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

# Plotting imports
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import textwrap


In [ ]:
# Import data
imported = pd.read_csv('SO test data.csv')
imported.head()

In [ ]:
# Rename columns for easier working with
ARRIVAL_COLUMN_NAME = 'Arrival Date/Time'
STREAM_COLUMN_NAME = "Stream"

imported = imported.rename({ARRIVAL_COLUMN_NAME: 'arrival_time', 
                            STREAM_COLUMN_NAME: "stream"}, 
                            axis=1)

# Add a column with no data that can just be used for counts
imported["dummy"] = 1

In [ ]:
imported.head()

In [ ]:
imported['arrival_time'] = pd.to_datetime(imported['arrival_time'], format="%d/%m/%Y %H:%M")

In [ ]:
imported['date'] = imported['arrival_time'].dt.date
imported['hour'] = imported['arrival_time'].dt.hour

In [ ]:
grouped = (
    imported.groupby(['date', 'hour', 'stream'])
    .count()[['dummy']]
    .reset_index()
)

grouped

In [ ]:
# Add a datetime column for the hour of arrival
# Ensure it's a datetime object rather than 
# a string
grouped['date_time_hour_start'] = (
    pd.to_datetime(grouped.date.astype('str') 
                   + ':' 
                   + grouped.hour.astype('str'), 
                   format='%Y-%m-%d:%H')
)

In [ ]:
# Select a subset of columns
grouped = grouped[['date_time_hour_start', 'stream', 'dummy']]

grouped

In [ ]:
# plot_list = []

# # Begin iterating through streams
# for stream in grouped.stream.unique():
#     # At the moment the name is hardcoded - this needs to change to iterate through given streams
#     stream_only = grouped[grouped['stream'] == stream]
#     # Get into correct format for Prophet
#     stream_only = (
#         stream_only.drop('stream', axis=1)
#         .rename({'date_time_hour_start': 'ds', 
#                     'dummy': 'y'}, axis=1)
#                     )

#     # log.info(stream_only.head(2))
    
#     # 1 week forecast
#     model = Prophet(interval_width=0.95)
#     model.add_country_holidays(country_name='England')
#     model.fit(stream_only)
#     future = model.make_future_dataframe(periods=24*7*8, 
#                                          freq='H', 
#                                          include_history=False)
#     fcst = model.predict(future)

#     fig  = plot_plotly(model, fcst)

#     # Update the x-axis range so we only display the future (i.e. the prediction),
#     # not the historic data, otherwise the period we are interested in is so small 
#     # as to not be visible
#     fig = fig.update_layout(xaxis_range=[stream_only.ds.max().to_pydatetime(), 
#                                             fcst.ds.max()])

#     plot_list.append({'title': stream, 'fig_json': fig.to_json(),
#                       'model': model, 'fcst': fcst})
#     print('Plot created for stream ' + str(stream))

# Stream objects

Create an object that can contain relevant information about streams: name, minutes per decision, and priority.

In [ ]:
class Stream:
    '''
    Object to contain required information about streams.
    '''
    def __init__(self,
                 name,
                 minutes_per_decision,
                 priority,
                 historic_data_all_streams,
                 twenty_four_hour_department=True,
                 open_hour=0,
                 close_hour=0,
                 weeks_to_forecast=8):
        '''
        Constructor

        Parameters:
        -----------
        name: str
            Name of the stream (e.g. majors, minors, resus)

        minutes_per_decision: int
            Number of minutes typically required to make a clinical 
            decision for each stream.

            This will be used to calculate the decision making time
            required from the number of attendances.

        priority: int
            Order in which the stream will be seen by clinicians who
            can float across different streams 

            Lower numbers = higher priority
            i.e. stream 1 will be seen first by clinicians so should
            be set for the most urgent stream

        historic_data_all_streams: pd.DataFrame
            dataframe with three columns:
            date_time_hour_start = DateTime column at hourly intervals
            stream = Stream
            dummy = Number of arrivals in that stream in that hour

        open_hour: str
            Opening time of the emergency department
            Decimal number of hours in 24 hour time (e.g. 7am would be 7,
            7:30am would be 7.5)
            Defaults to being 24 hour
            If not 24 hour, uses this time to cut off forecasting
            after this point, improving forecast accuracy

        close_hour: float
            Closing time of the emergency department
            Decimal number of hours in 24 hour time (e.g. 6pm would be 18,
            6:30pm would be 6.5)
            Defaults to being 24 hour
            If not 24 hour, uses this time to cut off forecasting
            after this point, improving forecast accuracy
        
        weeks_to_forecast: int
            Number of weeks to run the forecast for
        '''
    
        self.name = name
        self.minutes_per_decision = minutes_per_decision
        self.priority = priority

        self.weeks_to_forecast = weeks_to_forecast

        self.twenty_four_hour_department = twenty_four_hour_department

        # TODO: Add warnings and fail to create model if this contradicts
        # the twenty_four_hour_department attribute
        self.open_hour = open_hour
        self.close_hour = close_hour
        self.historic_data = historic_data_all_streams[historic_data_all_streams['stream'] == self.name]

    def fit_prophet_model_to_stream(self, interval_width=0.95,
                                    country='England'):
        '''
        Fits a Facebook Prophet forecasting model to hourly time
        series data and returns a forecast dataframe

        Params
        ------- 
        
        interval_width: float, default 0.95
            Value between 0 and 1 that will set the width of the 
            intervals. 
            0.95 = 95% confidence interval etc.

        country: string, default 'England'
            Country to include holidays for. 
            Expected options: one of 'England', 'Wales', 'Scotland', 
            'NorthernIreland', 'Ireland'

            All valid country names can be found here:
            https://github.com/dr-prodigy/python-holidays
        '''
        # Get into correct format for Prophet
        df = (
            self.historic_data.drop('stream', axis=1)
            .rename({'date_time_hour_start': 'ds', 
                        'dummy': 'y'}, axis=1)
                        )
        
        # 8 week forecast
        model = Prophet(interval_width=interval_width)
        model.add_country_holidays(country_name=country)
        model.fit(df)
        future = model.make_future_dataframe(
            periods=24 * 7 * self.weeks_to_forecast, 
            freq='H', 
            include_history=False
            )
        
        # If not a 24 hour department, filter
        # See docs here, section 'Data with regular gaps':
        # https://facebook.github.io/prophet/docs/non-daily_data.html
        # If this isn't done then the 'daily seasonality... [will be] unconstrained
        # for the rest of the day [where there is not historical data]'
        if not self.twenty_four_hour_department:
            future_excluding_closed = future.copy()
            # Filter out time before opening
            future_excluding_closed = (
                future_excluding_closed[future_excluding_closed['ds'].dt.hour > self.open_hour]
            )
            # Filter out time after closing
            future_excluding_closed = (
                future_excluding_closed[future_excluding_closed['ds'].dt.hour < self.close_hour]
            )
            forecast_df = model.predict(future_excluding_closed)
        else:
            forecast_df = model.predict(future)

        self.prophet_model = model
        self.forecast_df = forecast_df

    def add_decision_making_time_hourly(self):
        self.forecast_df['decision_making_time_hour'] = (
            self.forecast_df['yhat'] * self.minutes_per_decision
        )
        
        self.forecast_df['decision_making_time_hour_lower'] = (
            self.forecast_df['yhat_lower'] * self.minutes_per_decision
        )
        
        self.forecast_df['decision_making_time_hour_upper'] = (
            self.forecast_df['yhat_upper'] * self.minutes_per_decision
        )

    def get_metrics(self):
        # Have to use parallel='processes' if running in Deepnote
        # as the default 'none' calls on ipywidgets, which aren't supported
        # in Deepnote but can't be specifically turned off - this is a 
        # reasonable workaround
        n_days_covered = (
            (stream_majors.historic_data.date_time_hour_start.max() 
            - stream_majors.historic_data.date_time_hour_start.min())
            .days
        )

        self.df_cv = cross_validation(self.prophet_model,
                                 initial=f'{n_days_covered - 240}  days',
                                 period='60 days',
                                 horizon=f'{24 * 7 * self.weeks_to_forecast} hours',
                                 parallel="processes",)
        
        self.performance_metrics = performance_metrics(self.df_cv)


In [ ]:
# Initialise streams
stream_majors = Stream('Majors', 60, 2, grouped)
stream_minors = Stream('Minors', 30, 3, grouped)
stream_resus = Stream('Resus', 50, 1, grouped)

In [ ]:
stream_objects = [stream_majors, stream_minors, stream_resus]

In [ ]:
# Fit models
[stream_object.fit_prophet_model_to_stream() 
 for stream_object 
 in stream_objects]

## Consider the average hourly attendances per stream

In [ ]:
# Resample to ensure we have values for every hour
grouped_hs = grouped.copy()

In [ ]:
grouped_hs = (
    grouped_hs[grouped_hs['stream'].isin(['Majors', 'Minors', 'Resus'])]
    .pivot_table(values='dummy', 
                 columns='stream', 
                 index='date_time_hour_start')
    .fillna(0)
)

In [ ]:
grouped_hs = (
    grouped_hs.set_index(pd.to_datetime(grouped_hs.index))
)

In [ ]:
grouped_hs.head()

In [ ]:
grouped_hs = grouped_hs.resample('H').asfreq()

In [ ]:
plot_df = pd.melt(grouped_hs.reset_index(), id_vars='date_time_hour_start')

In [ ]:
px.box(plot_df, 
       x='stream', y='value', color='stream')

In [ ]:
px.histogram(plot_df, 
                x='value', color='stream', opacity=0.3, )

# Convert attendances to decision-making time required per hour

In [ ]:
# For any negative predictions, replace them with zero

for stream_object in stream_objects:
    for col in ['yhat', 'yhat_lower', 'yhat_upper']:
        stream_object.forecast_df[col] = (
            stream_object.forecast_df.apply(lambda x: x[col] if x[col] >=0 else 0, axis=1)
        )

In [ ]:
[stream_object.add_decision_making_time_hourly() 
 for stream_object 
 in stream_objects]

In [ ]:
stream_majors.forecast_df

# Class Definition

## Shift Type

In [ ]:
class ShiftType:
    def __init__(self,
                 name,
                 start_time,
                 end_time,
                 unavailability_1_start=None,
                 unavailability_1_end=None,
                 unavailability_2_start=None,
                 unavailability_2_end=None,
                 unavailability_3_start=None,
                 unavailability_3_end=None):
        '''
        Params:
        --------

        name: str
            Name of shift type
            (e.g. early, late, all day)

        start_time: str
            Time the shift begins
            Pass in the form HH:MM

        end_time: str
            Time the shift ends
            Pass in the form HH:MM

        unavailability_1_start: str (OPTIONAL)
            Time the first period of unavailability starts
            Pass in the form HH:MM

        unavailability_1_end: str (OPTIONAL)
            Time the first period of unavailability ends
            Pass in the form HH:MM

        unavailability_2_start: str (OPTIONAL)
            Time the second period of unavailability starts
            Pass in the form HH:MM            

        unavailability_2_end: str (OPTIONAL)
            Time the second period of unavailability ends
            Pass in the form HH:MM


        unavailability_3_start: str (OPTIONAL)
            Time the third period of unavailability starts
            Pass in the form HH:MM
            
        unavailability_3_end: str (OPTIONAL)
            Time the third period of unavailability ends
            Pass in the form HH:MM
        '''

        self.name = name
        self.name_plottable = name.replace('_', ' ').title()

        self.start_time = datetime.strptime(start_time,'%H:%M').time()
        self.end_time = datetime.strptime(end_time,'%H:%M').time()

        self.unavailability_1_start = self.try_datetime_parse(unavailability_1_start)
        self.unavailability_1_end = self.try_datetime_parse(unavailability_1_end)

        self.unavailability_2_start = self.try_datetime_parse(unavailability_2_start)
        self.unavailability_2_end = self.try_datetime_parse(unavailability_2_end)

        self.unavailability_3_start = self.try_datetime_parse(unavailability_3_start)
        self.unavailability_3_end = self.try_datetime_parse(unavailability_3_end)

    def try_datetime_parse(self, time_string):
        if time_string is not None:
            return datetime.strptime(time_string,'%H:%M').time()
        else:
            return None

    def decimal_time(self, time_of_interest):
        requested_time = getattr(self, time_of_interest)
        if requested_time is not None:
            return requested_time.hour + (requested_time.minute/60)
        else:
            return None

    def shift_type_dataframe(self):
        data = {
            'start_time': self.start_time,
            'end_time': self.end_time,
            'unavailability_1_start': self.unavailability_1_start,
            'unavailability_1_end': self.unavailability_1_end,
            'unavailability_2_start': self.unavailability_2_start,
            'unavailability_2_end': self.unavailability_2_end,
            'unavailability_3_start': self.unavailability_3_start,
            'unavailability_3_end': self.unavailability_3_end,
        }
        
        return pd.DataFrame.from_dict(
            data=data, 
            orient='index', 
            columns = [self.name]
            ) 

In [ ]:
def plot_shift_types(shift_type_list,
                    return_json=False):
    '''
    Outputs a plotly plot showing the 

    Params
    -------
    shift_type_list: list
        List of shift type objects
    '''

    # --- Set up plot --- #

    fig = go.Figure()

    # Work out how wide the plot needs to be by looking at the number
    # of shift types we have been given 
    plot_width = 3 * (len(shift_type_list)) + 1

    # Generate y axis labels
    timestrings = []
    t = datetime(2021, 1, 1, 0, 0, 0)
    while t < datetime(2021, 1, 2):
        timestrings.append(str(t.strftime('%H:%M')))
        t = t + timedelta(hours=1)
    time_labels = timestrings * 2
    time_labels = time_labels + ["00:00"]

    # Set axes properties
    fig.update_xaxes(range=[0, plot_width], 
                     showgrid=False, 
                     visible=False)
    # Set height of axes
    # Goes below 0 to allow for 
    fig.update_yaxes(range=[-26, 26], 
                     tickvals=list(range(-24, 25, 1)),
                     ticktext= time_labels)

    # --- Begin iterating through shift types --- #
    for i, shift_type in enumerate(shift_type_list):
        
        # Start by dealing with start and end times
        start_time_decimal_shift = shift_type.decimal_time('start_time')
        end_time_decimal_shift = shift_type.decimal_time('end_time')

        # If the start time is before the end time, 
        # implying a shift that doesn't span midnight
        if start_time_decimal_shift < end_time_decimal_shift:
            y0 = start_time_decimal_shift - 24 
            y1 = end_time_decimal_shift - 24
        
        # If the start time is after the end time, implying the 
        # shift spans midnight
        else: 
            y0 = end_time_decimal_shift 
            y1 = start_time_decimal_shift - 24

        fig.add_shape(type="rect",
            x0 = 3*(i+1) - 2,
            x1 = 3*(i+1) , 
            y0 = y0, 
            y1 = y1,
            line = {
                'color': "RoyalBlue",
                'width': 2,
            },
            fillcolor = "LightSkyBlue",
            name = shift_type.name
        )

        fig.add_annotation(
            x=3 * (i+1) - 1, 
            y=26,
            text='<br>'.join(textwrap.wrap(shift_type.name_plottable, width=10)),
            showarrow=False,
            yshift=10
            )

        # --- Now deal with break times --- #
        for j in ["unavailability_1", "unavailability_2", "unavailability_3"]:
            # Get the start and end time for the break as a decimal
            start_time_decimal_break = shift_type.decimal_time(f'{j}_start')
            end_time_decimal_break = shift_type.decimal_time(f'{j}_end')

            # Check whether a break is defined
            if (start_time_decimal_break is not None) and (end_time_decimal_break is not None):

                # Deal with breaks if shift doesn't span midnight
                if start_time_decimal_shift < end_time_decimal_shift:
                    y0 = start_time_decimal_break - 24 
                    y1 = end_time_decimal_break - 24

                # Deal with breaks if shift does span midnight
                else:                     
                    # If break start is bigger than a break end,
                    # implies the break spans midnight
                    if start_time_decimal_break > end_time_decimal_break:
                        y0 = start_time_decimal_break - 24 
                        y1 = end_time_decimal_break 
                    
                    # If start and end time both after midnight
                    elif (start_time_decimal_break < 12) and (end_time_decimal_break < 12):
                        y0 = start_time_decimal_break  
                        y1 = end_time_decimal_break

                    # If start and end time of break both before midnight
                    else: 
                        y0 = start_time_decimal_break - 24
                        y1 = end_time_decimal_break - 24

                # Plot the rectangle that represents the break
                fig.add_shape(type="rect",
                    x0 = 3*(i+1) - 2,
                    x1 = 3*(i+1) , 
                    y0 = y0, 
                    y1 = y1,
                    line = {
                        'color': "RoyalBlue",
                        'width': 2,
                    },
                    fillcolor = "LightGrey",
                    name = shift_type.name,
                    # marker_pattern_shape="/", 
                )

    # Add a line showing midnight between the two days
    fig.add_shape(
        type="line",
        xref="x", yref="y",
        x0=0, x1=plot_width,
        y0=0, y1=0, 
        line={
            'color': "DarkOrange",
            'width': 1,
            'dash': "dash"
        },
    )

    # Resize the plot
    fig.update_layout(
        autosize=False,
        width=600,
        height=800
        )

    if return_json:
        return fig.to_json()
    else:
        fig.show()
    

## Role Type

In [ ]:
class Role:
    def __init__(self,
                role_name,
                decisions_per_hour_per_stream):

        '''
        The role defines the decision-making capabilities of a 
        particular class of decision maker

        e.g. a Role could be 'Consultant majors'

        You may have >1 individual with the same role in an ED
        

        Params:
        -------

        role_name: str
            Name of role
            Examples: Cons Resus, Cons Majors, Cons Minors

        decisions_per_hour_per_stream: list of dicts
            List of dicts in the following format
            {'stream': str, 'decisions_per_hour': float}
            Where stream is the stream name
            Decisions per hour is 

            If a resource is able to make decisions for multiple streams,
            then the list should contain multiple dictionaries

        '''
    
        self.role_name = role_name
        self.decisions_per_hour_per_stream = decisions_per_hour_per_stream 

## Rota Entry

In [ ]:
class RotaEntry:
    '''
    Object defining a week's worth of rota for a single
    individual
    '''

    def __init__(self,
                 role,
                 core=True,
                 name=None,
                 prev_week=None,
                 monday=None,
                 tuesday=None,
                 wednesday=None,
                 thursday=None,
                 friday=None,
                 saturday=None,
                 sunday=None):
        '''
        role: RoleType object
            What role the rota entry relates to. 
            RoleType objects determine decisions per hour.

        core: boolean
            Whether a resource should be considered as core.
            False = resource is ad-hoc.


        name: str
            String giving name of resource e.g. if preferring to 
            work with actual names of individuals

        prev_week: ShiftType object 

        monday: ShiftType object

        tuesday: ShiftType object

        wednesday: ShiftType object

        thursday: ShiftType object

        friday: ShiftType object
        
        saturday: ShiftType object
        
        sunday: ShiftType object
        '''
        self.role = role
        self.core = core
        self.name = name

        self.prev_week = prev_week
        self.monday = monday
        self.tuesday = tuesday
        self.wednesday = wednesday
        self.thursday = thursday
        self.friday = friday
        self.saturday = saturday
        self.sunday = sunday

## Rota Template

In [ ]:
class RotaTemplate:
    def __init__(self,
                 name,
                 rota,
                 start_date):
        '''

        Params:
        -------
        
        name: str
            Name for rota template

        rota: list
            List of RotaEntry objects

        '''
        self.name = name
        self.rota = rota
        self.start_date = datetime.strptime(start_date, "%d/%m/%Y").date()
        self.end_date = self.start_date + timedelta(days=7)

# ED Definition

## Set up shift types

In [ ]:
all_day = ShiftType(name='all_day',
                  start_time='08:00',
                  end_time='21:00',
                  unavailability_1_start='12:00',
                  unavailability_1_end='13:00')

early = ShiftType(name='Early',
                  start_time="06:00",
                  end_time="18:00",)

late = ShiftType(name='late',
                  start_time='10:00',
                  end_time='22:00',)

overnight= ShiftType(name='overnight',
                  start_time='21:00',
                  end_time='07:00',)

half_hour_test= ShiftType(name='half_hour_test',
                  start_time='21:30',
                  end_time='07:30',
                  unavailability_1_start='22:30',
                  unavailability_1_end='22:45',
                  unavailability_2_start='23:30',
                  unavailability_2_end='00:45',
                  unavailability_3_start='04:30',
                  unavailability_3_end='05:45'
                )

shift_types = [all_day, early, late, overnight, half_hour_test]

In [ ]:
plot_shift_types(shift_types)

In [ ]:
# Check size of json object to see size of response that will be sent
# following API request
sys.getsizeof(plot_shift_types(shift_types, return_json=True)) / 1000

## Set up role types

In [ ]:
cons_resus = Role('Cons Resus', {'Resus': 1})
cons_majors = Role('Cons Majors', {'Majors': 2})
cons_minors = Role('Cons Minors', {'Minors': 2.5})
cons_flex = Role('Cons Flex', {'Majors': 1, 'Resus': 1})

fy2_resus = Role('FY2 Resus', {'Resus': 1})
fy2_majors = Role('FY2 Majors', {'Majors': 1})
fy2_minors = Role('FY2 Minors', {'Minors': 2})
fy2_flex = Role('FY2 Flex', {'Majors': 1, 'Resus': 1})

enp_majors = Role('ENP Majors', {'Majors': 0.5})
enp_minors = Role('ENP Minors', {'Minors': 1})
enp_flex = Role('ENP Flex', {'Majors': 0.5, 'Minors': 1})

band_5 = Role('Band 5', {'Minors': 1})

## Set up rota entries

In [ ]:

v1 = RotaTemplate(
    name = "v1",
    rota = [
        RotaEntry(cons_resus, True, monday=overnight, tuesday=overnight, wednesday=overnight, thursday=all_day, friday=all_day),
        RotaEntry(cons_majors, True, monday=overnight, tuesday=overnight, wednesday=overnight, thursday=overnight, friday=overnight),
        RotaEntry(cons_minors, True, prev_week=overnight, monday=early, tuesday=early, wednesday=early, thursday=early, friday=early),
        RotaEntry(cons_majors, False, monday=all_day, tuesday=all_day, wednesday=all_day, saturday=all_day, sunday=all_day),
        RotaEntry(fy2_resus, True, prev_week=overnight, monday=overnight, tuesday=overnight, wednesday=overnight, thursday=overnight),
        RotaEntry(fy2_majors, True, monday=all_day, tuesday=all_day, wednesday=all_day, thursday=all_day, sunday=all_day),
        RotaEntry(fy2_minors, True, wednesday=overnight, friday=all_day, saturday=all_day, sunday=all_day)
    ],
    start_date = "19/07/2021",
)

### Compare core and adhoc resources

In [ ]:
resources = []

for individual_resource in v1.rota:
    resources.append({'role': individual_resource.role.role_name,
    'core': individual_resource.core})

resources_df = pd.DataFrame.from_dict(resources)
resources_df['count'] = 1

In [ ]:
resources_df_core_adhoc_count = (
    resources_df.groupby(['role', 'core'])
    .count()
    .reset_index()
)

In [ ]:
# resources_df_core_adhoc_count.pivot_table('role')

### Calculate available capacity from rota



# Assess forecast quality

In [ ]:
def plot_metrics_streams(stream_objects,
                         metric_of_interest,
                         figsize=(14,10),
                         sharey=True):
    '''
    Given a list of stream objects, output plots showing
    the value of a metric for assessing forecast quality
    over a range of forecast horizons 

    Parameters
    ---------

    stream_objects: list 
        list containing objects of type Stream 
        that have been fully initialised 

    metric_of_interest: string
        One of mae, mape, mse, rmse, mdape, smape, coverage 

    figsize: tuple
        Size of matplotlib plot
    '''

    fig, ax = plt.subplots(nrows=int(math.ceil(len(stream_objects)/3)), 
                           ncols=3, figsize=figsize, sharey=sharey)

    for i in range(len(stream_objects)):
        plot_cross_validation_metric(
            stream_objects[i].df_cv, 
            metric=metric_of_interest,
            ax=ax.flatten()[i]
            ) 
        # Set title for given axis
        ax.flatten()[i].title.set_text(stream_objects[i].name)
    
    fig.suptitle(metric_of_interest)

In [ ]:
# Test model quality
# [stream_object.get_metrics() 
#  for stream_object 
#  in stream_objects]

In [ ]:
# plot_metrics_streams(stream_objects, "coverage")

In [ ]:
# plot_metrics_streams(stream_objects, "mae")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a4cc3d8f-976a-44f2-a841-ef4378fe07d2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>